# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [116]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [117]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
from mpmath import mpf
import mpmath

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: @reh_trinanes


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [118]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @reh_trinanes

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [119]:
#Produto escolhido:
produto = 'Macbook'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [120]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [121]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [122]:
dados = pd.read_excel('Projeto2_macbook copy.xlsx')

dados = dados.loc[:,["Treinamento", "Classificação"]]
vc = dados.Classificação.value_counts(True)
prob_tweet_relevante = vc.loc["Relevante"]
#Probabilidade do tweet ser irrelevante
prob_tweet_irrelevante = vc.loc["Irrelevante"]


In [123]:
#para deixar todos os tweets como se fossem um único tweet "gigante"
tweet_concatenado = np.sum(dados.Treinamento + " ") 

#irrelevante
tweet_concatenado_irrel = np.sum(dados[dados.Classificação=="Irrelevante"].Treinamento + " ")

#relevante
tweet_concatenado_rel = np.sum(dados[dados.Classificação=="Relevante"].Treinamento + " ")

In [124]:
#tirando a pontuação
import string

exclude = set(string.punctuation)  #pontuation pega todas as pontuaçoes 
def SemPontuacao(tweet_concatenado):
    tweet_concatenado = tweet_concatenado.split() #para cada palavra do tweet gigante virar um tweet de palavra única
    lista_cada_palavra = []
    for s in tweet_concatenado:
        for i in s:
            if i in exclude:
                s = s.replace(i,'') 
        lista_cada_palavra.append(s) #volta a ser um tweet gigante
    return lista_cada_palavra   
    
concatenado_irrel = SemPontuacao(tweet_concatenado_irrel)
concatenado_rel = SemPontuacao(tweet_concatenado_rel)


In [125]:
#tweet concantenado em Series
palavras_irrel = pd.Series(concatenado_irrel)
palavras_rel = pd.Series(concatenado_rel)
palavras_totais = pd.Series(concatenado_irrel + concatenado_rel)

In [126]:
#para saber quantas palavras diferentes existem entre relevantes e irrelevantes
palavras_dif = len(palavras_totais.value_counts())

palavras_dif_rel = len(palavras_rel.value_counts())
palavras_dif_irrel = len(palavras_irrel.value_counts())


#cada palavra e quantas vezes essa palavra apareceu
qntd_palavras_dif_rel = (palavras_rel.value_counts())
qntd_palavras_dif_irrel = (palavras_irrel.value_counts())
qntd_palavras_dif = (palavras_totais.value_counts())

qtd_relevante = len(dados[dados.Classificação=="Relevante"])
qtd_irrelevante = len(dados[dados.Classificação=="Irrelevante"])
qtd_total_tweets = qtd_relevante + qtd_irrelevante

In [127]:
#para calcular a propabilidade de cada palavra aparecer em Relevante
lista_palavras_rel = []
lista_prob_rel = []
for palavra in qntd_palavras_dif_rel.index:
    
    prob_palavra_rel = ( qntd_palavras_dif_rel[palavra] + 1 ) / ( len(palavras_rel) + palavras_dif_rel)
    lista_palavras_rel.append(palavra)
    lista_prob_rel.append(prob_palavra_rel)
    print("A palavra ", palavra, " aparece ", qntd_palavras_dif_rel[palavra], " vezes")
    print ("A probabilidade dela aparecer em Relevante é:", prob_palavra_rel )
    print ("      ")


A palavra  macbook  aparece  41  vezes
A probabilidade dela aparecer em Relevante é: 0.046511627907
      
A palavra  que  aparece  22  vezes
A probabilidade dela aparecer em Relevante é: 0.0254706533776
      
A palavra  o  aparece  15  vezes
A probabilidade dela aparecer em Relevante é: 0.0177187153931
      
A palavra  um  aparece  14  vezes
A probabilidade dela aparecer em Relevante é: 0.0166112956811
      
A palavra  tecmundo  aparece  14  vezes
A probabilidade dela aparecer em Relevante é: 0.0166112956811
      
A palavra  comprar  aparece  13  vezes
A probabilidade dela aparecer em Relevante é: 0.015503875969
      
A palavra  de  aparece  13  vezes
A probabilidade dela aparecer em Relevante é: 0.015503875969
      
A palavra  da  aparece  12  vezes
A probabilidade dela aparecer em Relevante é: 0.0143964562569
      
A palavra  em  aparece  12  vezes
A probabilidade dela aparecer em Relevante é: 0.0143964562569
      
A palavra  por  aparece  11  vezes
A probabilidade dela apar

      
A palavra  2000  aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra  descobri  aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra  mil  aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra  500  aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra    aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra  tá  aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra  bom  aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra  não  aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra  casa  aparece  2  vezes
A probabilidade dela aparecer em Relevante é: 0.00332225913621
      
A palavra  com  aparece  2  vezes
A probabilidade dela

A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  lixo  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  zero  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  graças  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  tive  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  qual  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  deu  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  razer  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  ontem  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.00221483942414
      
A palavra  volto  aparece  1  vezes
A probabilidade dela aparecer em Relevante é: 0.002

In [128]:
#para calcular a propabilidade de cada palavra aparecer em Irrelevante
lista_palavras_irrel = []
lista_prob_irrel = []
for palavra in qntd_palavras_dif_irrel.index:
    
    prob_palavra_irrelevante = ( qntd_palavras_dif_irrel[palavra] + 1 ) / ( len(palavras_irrel) + palavras_dif_irrel)
    lista_palavras_irrel.append(palavra)
    lista_prob_irrel.append(prob_palavra_irrelevante)
    print("A palavra ", palavra, " aparece ", qntd_palavras_dif_irrel[palavra], " vezes")
    print ("A probabilidade dela aparecer em Irrelevante é:", prob_palavra_irrel )
    print ("      ")

A palavra  macbook  aparece  250  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  um  aparece  125  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  o  aparece  109  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  de  aparece  108  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  a  aparece  94  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  e  aparece  78  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  que  aparece  66  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  rt  aparece  58  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  do  aparece  57  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  é  aparece  55  vezes
A probabilidade dela aparecer em 

A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  comprei  aparece  6  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  n  aparece  6  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  novo  aparece  6  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  para  aparece  6  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  vi  aparece  5  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  venda  aparece  5  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  baixar  aparece  5  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  nesse  aparece  5  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  500gb  aparece  5  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657

A palavra  mercado  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  mostrando  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  meninas  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  pai  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  aquele  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  capa  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  sonho  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  vergonha  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  2015  aparece  2  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  indo  aparece  2  vezes
A probabilid

      
A palavra  mil  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  análise  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  senhorjon  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  filiar  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  levantar  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  anti  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  dif  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  email  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  pirei  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  interface  aparece  1  vezes
A p

A palavra  may  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  enviaram  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  core2duo  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  estudar  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  queriam  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  marcelobechler  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  vintage  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  trabalhar  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  peças  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  vitorgrs  aparece  1 

A palavra  qualquer  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  oslecmac74  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  provar  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  j0lha  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  capitalismo  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  partitura…  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  trabalham  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  usbc  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  heybabss  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  filipekids  aparec

A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  arturei  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  paralela  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  roadsec  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  thug  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  young  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  jujulemes  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  28ghz  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  autocorrect  aparece  1  vezes
A probabilidade dela aparecer em Irrelevante é: 1.00017943657
      
A palavra  dotô  aparece  1  vezes
A probabilidade dela aparecer em Irrelev

In [129]:
#Criando um dicionário com as palavras e suas probabilidades DEF   
dicionario_relevantes = dict()
dicionario_irrelevantes = dict()
def Dic(lista_palavra,lista_prob,dic):
    for i in range(len(lista_palavra)):
        dic[lista_palavra[i]]= lista_prob[i]
    return(dic)

prob_p_relevante = Dic(lista_palavras_rel,lista_prob_rel,dicionario_relevantes)

prob_p_irrelevante = Dic(lista_palavras_irrel,lista_prob_irrel,dicionario_irrelevantes)

### Analisando a aba Teste

In [130]:
#para pegar os dados da aba Teste
dados_teste = pd.read_excel('Projeto2_Macbook copy.xlsx', "Teste", sep=',')

dados_teste

,Teste,Classificação
0,"reparo, troca do display para o macbook air 11...",Irrelevante
1,rt @xxpjl: alguém podia me dar um macbook de p...,Irrelevante
2,tenho que comprar um macbook pra produzir no a...,Irrelevante
3,tópicos do mm fórum: formatando macbook roubad...,Irrelevante
4,@greyslogia olha o macbook dela,Irrelevante
5,não vejo a hora de ter meu macbook. pra para c...,Irrelevante
6,preciso de um macbook \nalguém?,Irrelevante
7,@dougdiskin @grigio do macbook não pq é uma de...,Irrelevante
8,tô afim de comprar um macbook,Irrelevante
9,triste ao saber que não sou hacker nem tenho u...,Relevante


In [131]:
lista_teste = dados_teste.Teste

#Tirando e pontuação e separando cada tweet em uma lista e depois, cada palavra desse tweet dentro dessa lista

exclude = set(string.punctuation)  #pontuation pega todas as pontuaçoes 

def SemPontuacao(tweet_gigante):
    lista_cada_palavra_do_tweet = []
    for g in range(len(tweet_gigante)):
        lista_tweets_splitados = tweet_gigante[g].split()
        lista_tweets = []

        for f in lista_tweets_splitados:
            for i in f:
                if i in exclude:
                    f = f.replace(i,'') 
                
            lista_tweets.append(f)
        lista_cada_palavra_do_tweet.append(lista_tweets)
                
    return lista_cada_palavra_do_tweet   

lista_palavras_tweet = SemPontuacao (lista_teste)

In [132]:
#funão que calcula a probabilidade da frase ser relevante ou irrelevante 
def Prob_tweet(listafrase,P_P,P_,T_):
    listaP = []
    for frase in listafrase:
        P_frase = mpmath.mpf(1.000)
        for p in frase:
            if p in P_P:
                P_frase = P_frase * P_P[p] * P_
            else:
                P_frase = mpmath.mpf(1)/(T_+palavras_dif_rel) * P_frase * P_
        listaP.append(P_frase)
    return listaP


#cria lista sem pontuação
teste_cada_tweet = SemPontuacao(lista_teste)

#Cálculo da probabilidade do tweet ser neutro
teste_prob_irrelevante = Prob_tweet(teste_cada_tweet, prob_p_irrelevante ,prob_tweet_irrelevante ,qtd_irrelevante)

#Cálculo da probabilidade do tweet ser relevante
teste_prob_relevante = Prob_tweet(teste_cada_tweet, prob_p_relevante, prob_tweet_relevante, qtd_relevante)



In [133]:
#comparar a lista de probabilidade feita no teste, e ver qual prevalece, se é neutro ou relevante
def Comparador(teste_prob_irrelevante,teste_prob_relevante):
    resultado = []
    for i in range(len(teste_prob_irrelevante)):
        if teste_prob_irrelevante[i] > teste_prob_relevante[i]:
            resultado.append('Irrelevante')
        elif teste_prob_irrelevante[i] < teste_prob_relevante[i]:
            resultado.append('Relevante')
    return resultado

#lista de respostas do aplicativo
resp = Comparador(teste_prob_irrelevante, teste_prob_relevante)

#len(resultado_teste)

In [134]:
data = {'Tweets': teste_cada_tweet, 'Classificação': dados.Classificação, 'Classificação': resp}

In [135]:
tabela_classificador = pd.DataFrame(data, columns = ['Tweets','Classificação'])

tabela_classificador

,Tweets,Classificação
0,"[reparo, troca, do, display, para, o, macbook,...",Irrelevante
1,"[rt, xxpjl, alguém, podia, me, dar, um, macboo...",Irrelevante
2,"[tenho, que, comprar, um, macbook, pra, produz...",Irrelevante
3,"[tópicos, do, mm, fórum, formatando, macbook, ...",Irrelevante
4,"[greyslogia, olha, o, macbook, dela]",Irrelevante
5,"[não, vejo, a, hora, de, ter, meu, macbook, pr...",Irrelevante
6,"[preciso, de, um, macbook, alguém]",Irrelevante
7,"[dougdiskin, grigio, do, macbook, não, pq, é, ...",Irrelevante
8,"[tô, afim, de, comprar, um, macbook]",Irrelevante
9,"[triste, ao, saber, que, não, sou, hacker, nem...",Irrelevante


In [136]:
#para pegar os dados da aba Teste
dados_teste = pd.read_excel('Projeto2_Macbook copy.xlsx', "Teste", sep=',')

#dados_teste

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [142]:
dados_teste.loc[(dados_teste.Classificação == "Relevante") & (tabela_classificador.Classificação == "Relevante"),"CLASS"]= "Positivo Verdadeiro" 
dados_teste.loc[(dados_teste.Classificação == "Irrelevante") & (tabela_classificador.Classificação == "Irrelevante"),"CLASS"]= "Negativo Verdadeiro"                            
dados_teste.loc[(dados_teste.Classificação == "Relevante") & (tabela_classificador.Classificação == "Irrelevante"),"CLASS"]= "Negativo Falso"                            
dados_teste.loc[(dados_teste.Classificação == "Irrelevante") & (tabela_classificador.Classificação == "Relevante"),"CLASS"]= "Positivo Falso"                            

dados_teste

,Teste,Classificação,CLASS
0,"reparo, troca do display para o macbook air 11...",Irrelevante,Negativo Verdadeiro
1,rt @xxpjl: alguém podia me dar um macbook de p...,Irrelevante,Negativo Verdadeiro
2,tenho que comprar um macbook pra produzir no a...,Irrelevante,Negativo Verdadeiro
3,tópicos do mm fórum: formatando macbook roubad...,Irrelevante,Negativo Verdadeiro
4,@greyslogia olha o macbook dela,Irrelevante,Negativo Verdadeiro
5,não vejo a hora de ter meu macbook. pra para c...,Irrelevante,Negativo Verdadeiro
6,preciso de um macbook \nalguém?,Irrelevante,Negativo Verdadeiro
7,@dougdiskin @grigio do macbook não pq é uma de...,Irrelevante,Negativo Verdadeiro
8,tô afim de comprar um macbook,Irrelevante,Negativo Verdadeiro
9,triste ao saber que não sou hacker nem tenho u...,Relevante,Negativo Falso


In [146]:
#comparação entre classificadores
dados_teste.CLASS.value_counts(True)*100


Negativo Verdadeiro    84.333333
Negativo Falso         13.000000
Positivo Verdadeiro     2.666667
Name: CLASS, dtype: float64

In [147]:
#porcentagem do meu classificador
dados_teste.Classificação.value_counts(True)*100

Irrelevante    84.333333
Relevante      15.666667
Name: Classificação, dtype: float64

In [148]:
#porcentagem do classificador Naive Bayes
tabela_classificador.Classificação.value_counts(True)*100

Irrelevante    97.333333
Relevante       2.666667
Name: Classificação, dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


A partir dos dados obtidos pela minha própria classificação a cerca dos tweets sobre Macbook em conjunto com a classificação pelo Naive Bayes, cheguei à conclusão de que o classificador não é de extrema eficiência, porém tem uma margem de erro relativamente pequena: 13%

** Meu classificador**
* Relevante: 84.3 %
* Irrelevante: 15.6 %

**Classificador Naive Bayes**
  
* Relevante: 97.3 %
* Irrelevante: 2.6 %
  
Ademais, o classificador não entende dupla negação e sarcasmo, tornando-o ainda mais ineficiente.

Meu projeto deve continuar sendo financiado pois há uma grande possibilidade de melhoria do classificador com mais testes, com uma pesquisa mais extensa e características mais específicas, assim podendo diminuir e até eliminar as disparidades entre o que queremos que o classificador faça e o que ele realmente faz.